In [ ]:
import os
import glob
import collections
import numpy as np
import tensorflow as tf
import cv2
import keras

from tensorflow.keras.models import load_model
from io import BytesIO
from tensorflow.keras.models import load_model



In [ ]:
#model_1 link:
'''
https://drive.google.com/file/d/1mH7HknToTEIu4mcHpZHTU-1XBnAYmEPa/view?usp=sharing
'''
#model_2 link:
'''
https://drive.google.com/file/d/1-NAJfiybE2_ElioP8tvU-DyE1r3K21mn/view?usp=sharing
'''

In [ ]:
def download_folder(link):
  '''
  download your test data innto the enviroment
  '''
  path= ''

  return path

In [ ]:
def submit(folder_link):
  model_prediction=[]
  '''
  Return:
  model_prediction -> list of zeros and ones based on the prediction for each video in the folder (0 -> not shop lifter | 1-> shop lifter)

  Args:
  folder_link -> link for a folder has some videos to predict

  '''

  # write your code here
  # load your model weights
  # loop on all the videos in the folder_link
  # append the predictions to the list model_prediction

  model_1 , model_2 = load_my_model()


  folder_path = download_folder(folder_link)

  files = [f for f in os.listdir(folder_path) if f.endswith(('.mp4', '.avi', '.mkv'))]
  for video_file in folder_path:
      video_path = os.path.join(folder_path, video_file)
      model_prediction.append(predict_one(model_1, model_2, video_path))



  # Note : the returned predictions should be 0 or 1 (0 -> not shop lifter | 1-> shop lifter)





  #---------------------------------------------------------------------------------------------------------


  return model_prediction

In [ ]:
def single_video_preprocessing(video_path, SEQUENCE_LENGTH ):
  '''
  this function preprocess only one video
  video path >> is a path to the video dowwnloaded into the working enviroment

  sequence lenght >> is the number of frames it is origanally 20

  return type is alist of shape (20, 64, 64, 3)
)
  '''

    # Initialize the VideoCapture object to read from the video file.
  video_reader = cv2.VideoCapture(video_path)

   # Get the width and height of the video
  original_video_width = int(video_reader.get(cv2.CAP_PROP_FRAME_WIDTH))
  original_video_height = int(video_reader.get(cv2.CAP_PROP_FRAME_HEIGHT))
    # Declare a list to store video frames we will extract.
  frames_list = []

   # Initialize a variable to store the predicted action being performed in the video.
  predicted_class_name = ''

   # Get the number of frames in the video.
  video_frames_count = int(video_reader.get(cv2.CAP_PROP_FRAME_COUNT))
  video_fps = video_reader.get(cv2.CAP_PROP_FPS)

  skip_frames_window = max(int(video_frames_count/SEQUENCE_LENGTH),1)

  for frame_counter in range(SEQUENCE_LENGTH):
      # Set the current frame position of the video.
    video_reader.set(cv2.CAP_PROP_POS_FRAMES, frame_counter * skip_frames_window)
      # Read a frame.
    success, frame = video_reader.read()
      # Check if frame is not read properly then break the loop.
    if not success:
      break

     # Resize the Frame to fixed Dimensions.
    resized_frame = cv2.resize(frame, (64, 64))

     # Normalize the resized frame by dividing it with 255 so that each pixel value then lies between 0 and 1.
    normalized_frame = resized_frame / 255

     # Appending the pre-processed frame into the frames list
    frames_list.append(normalized_frame)

  video_reader.release()

  return frames_list

In [ ]:
def load_my_model():
  ##! gdown --id 1LDyrBdoS5CIsBNPMNzvoTZKpYqe-wgXL
  #model_1 link (must be 1)
  model_1 = tf.keras.saving.load_model("/content/convlstm_model_1__Date_Time_2024_03_0812_15_10_Loss_1.6097403764724731__Accuracy_0.6610169410705566.h5")
  #model_2 link (must be 2)
  model_2 = tf.keras.saving.load_model("/content/convlstm_model_2__Date_Time_2024_03_0811_58_22_Loss_0.2769467234611511__Accuracy_0.8717948794364929.h5")
  return model_1,model_2

In [ ]:
def predict_one(model_1, model_2, file_path):
  input = []
  input.append(single_video_preprocessing(file_path, 20))
  input = np.array(input)
  mymodel_1 = model_1
  mymodel_2 = model_2

  prob1 = mymodel_1.predict(input)
  prob2 = mymodel_2.predict(input)
  prob1 = prob1.tolist()
  prob2 = prob2.tolist()

  sl_1 = prob1[0][0]
  normal = prob1[0][1]
  tb_1 = prob1[0][2]

  sl_2 = prob2[0][0]
  sh = prob2[0][1]
  tb_2 = prob2[0][2]

  threshold = 0.3

  if normal_1 < threshold: #normal is less than threshold (go to the second model)

    if sl_1 > threshold: #see & let in the first model
      if sl_2 > threshold:   #see & let in the 2nd model too (normal)
        return 0  #print("normal sl")
      else:   # 2nd model has take & buy or shoplifter
        return 1 #print("shoplift sl1")

    if tb_1 > threshold:  #take & buy happened in the first model
      if tb_2 > threshold: # take & buy also (normal)
        return 0   #print("normal tb")
      else:     # 2nd model is something different than we know
        return 1  #print("shoplift tb1")

  else: #normal is above threshold in the first model
    return 0  #print("normal")


(1, 15, 64, 64, 3)

1/1 [==============================] - 0s 37ms/step


1